# Importing packages and libraries

In [298]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import scipy.cluster.hierarchy as hc
from sklearn.preprocessing import normalize
from scipy.cluster.hierarchy import fcluster
from sklearn.cluster import KMeans as km
from sklearn.preprocessing import normalize
from yellowbrick.cluster import KElbowVisualizer

C:\Users\yetur\Anaconda3\lib\site-packages\seaborn\rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
C:\Users\yetur\Anaconda3\lib\site-packages\setuptools\_distutils\version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


# Part I: Analyzing Transactions

# Importing Dataset to jupyter notebook

In [23]:
df=pd.read_csv(r"transactions.csv")
df.head()

,Transaction,Product
0,12359,Candy Bar
1,12362,Pain Reliever
2,12362,Pain Reliever
3,12365,Toothpaste
4,12371,Bow


<h><b>1. Read in the data and generate a file in which every row represents a transaction, with True identifying items
that were part of that transaction, and False identifying items that were not (as in the example from class).
    Name the file group04transactions01.csv, where xx is your group number.</b></h>

In [25]:
df=df.drop_duplicates()
df=pd.get_dummies(df, prefix='',prefix_sep='')
df=df.groupby(['Transaction']).sum().reset_index()
df.set_index('Transaction',inplace=True)
df = df.astype(bool)
df.to_csv('group04transaction01.csv')

,Bow,Candy Bar,Deodorant,Greeting Cards,Magazine,Markers,Pain Reliever,Pencils,Pens,Perfume,Photo Processing,Prescription Med,Shampoo,Soap,Toothbrush,Toothpaste,Wrapping Paper
Transaction,,,,,,,,,,,,,,,,,
12359,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
12362,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
12365,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
12371,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
12380,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221845,False,False,False,True,True,False,False,False,True,False,True,False,False,False,False,True,False
1221854,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
1221857,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


<h><b>2. Identify the frequent itemsets using a minimum support threshold of 1%.</b></h>

In [28]:
fi = apriori(df, min_support = 0.01, use_colnames = True)
print(len(fi))
fi

40


,support,itemsets
0,0.054645,(Bow)
1,0.171005,(Candy Bar)
2,0.146885,(Greeting Cards)
3,0.241305,(Magazine)
4,0.026700,(Pain Reliever)
5,0.134925,(Pencils)
6,0.143575,(Pens)
7,0.089960,(Perfume)
8,0.058480,(Photo Processing)
9,0.014505,(Prescription Med)


<h><b>3. Identify all association rules with a minimum confidence of 10%.</b></h>

In [42]:
rules = association_rules(fi, metric="confidence", min_threshold=0.1)
print(len(rules))
rules

50


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bow),(Toothbrush),0.054645,0.067350,0.011340,0.207521,3.081236,0.007660,1.176877
1,(Toothbrush),(Bow),0.067350,0.054645,0.011340,0.168374,3.081236,0.007660,1.136755
2,(Candy Bar),(Greeting Cards),0.171005,0.146885,0.043660,0.255314,1.738191,0.018542,1.145604
3,(Greeting Cards),(Candy Bar),0.146885,0.171005,0.043660,0.297239,1.738191,0.018542,1.179626
4,(Candy Bar),(Magazine),0.171005,0.241305,0.040535,0.237040,0.982325,-0.000729,0.994410
5,(Magazine),(Candy Bar),0.241305,0.171005,0.040535,0.167982,0.982325,-0.000729,0.996367
6,(Candy Bar),(Pencils),0.171005,0.134925,0.033015,0.193065,1.430903,0.009942,1.072050
7,(Pencils),(Candy Bar),0.134925,0.171005,0.033015,0.244691,1.430903,0.009942,1.097558
8,(Candy Bar),(Toothpaste),0.171005,0.160425,0.039780,0.232625,1.450053,0.012347,1.094087
9,(Toothpaste),(Candy Bar),0.160425,0.171005,0.039780,0.247966,1.450053,0.012347,1.102338


<h><b>4. Rules with highest lift, Using the results from the previous questions, this lift
value was calculated for one of the rules with highest lift.</b></h>

In [41]:
rules1 = rules.sort_values(['lift'], ascending =[False])
rules1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
26,(Perfume),(Toothbrush),0.089960,0.067350,0.021820,0.242552,3.601370,0.015761,1.231306
27,(Toothbrush),(Perfume),0.067350,0.089960,0.021820,0.323979,3.601370,0.015761,1.346172
1,(Toothbrush),(Bow),0.067350,0.054645,0.011340,0.168374,3.081236,0.007660,1.136755
0,(Bow),(Toothbrush),0.054645,0.067350,0.011340,0.207521,3.081236,0.007660,1.176877
28,"(Candy Bar, Magazine)",(Greeting Cards),0.040535,0.146885,0.016665,0.411126,2.798966,0.010711,1.448723
31,(Greeting Cards),"(Candy Bar, Magazine)",0.146885,0.040535,0.016665,0.113456,2.798966,0.010711,1.082253
43,"(Pencils, Toothpaste)",(Candy Bar),0.024560,0.171005,0.011390,0.463762,2.711980,0.007190,1.545947
30,"(Magazine, Greeting Cards)",(Candy Bar),0.036335,0.171005,0.016665,0.458649,2.682078,0.010452,1.531344
47,"(Magazine, Toothpaste)",(Greeting Cards),0.031665,0.146885,0.011945,0.377230,2.568202,0.007294,1.369873
40,"(Magazine, Toothpaste)",(Candy Bar),0.031665,0.171005,0.013720,0.433286,2.533762,0.008305,1.462810


<t>We have two similar results for lift in the values dataframe hence we consider the one with highest confidence.</t>

In [43]:
rules2 = rules.sort_values(['lift','confidence'], ascending =[False,False], ignore_index=True)
rules2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Perfume),(Toothbrush),0.089960,0.067350,0.021820,0.242552,3.601370,0.015761,1.231306
1,(Toothbrush),(Perfume),0.067350,0.089960,0.021820,0.323979,3.601370,0.015761,1.346172
2,(Bow),(Toothbrush),0.054645,0.067350,0.011340,0.207521,3.081236,0.007660,1.176877
3,(Toothbrush),(Bow),0.067350,0.054645,0.011340,0.168374,3.081236,0.007660,1.136755
4,"(Candy Bar, Magazine)",(Greeting Cards),0.040535,0.146885,0.016665,0.411126,2.798966,0.010711,1.448723
5,(Greeting Cards),"(Candy Bar, Magazine)",0.146885,0.040535,0.016665,0.113456,2.798966,0.010711,1.082253
6,"(Pencils, Toothpaste)",(Candy Bar),0.024560,0.171005,0.011390,0.463762,2.711980,0.007190,1.545947
7,"(Magazine, Greeting Cards)",(Candy Bar),0.036335,0.171005,0.016665,0.458649,2.682078,0.010452,1.531344
8,"(Magazine, Toothpaste)",(Greeting Cards),0.031665,0.146885,0.011945,0.377230,2.568202,0.007294,1.369873
9,"(Magazine, Toothpaste)",(Candy Bar),0.031665,0.171005,0.013720,0.433286,2.533762,0.008305,1.462810


In [44]:
lift_value=rules2.iloc[0,5]/rules2.iloc[0,3]
print(lift_value)

3.6013696427976076


<h><b>5. For the same rule, leverage and conviction were obtained.</b></h>

<t> There is a covariance of 0.015761 between antecedent and consequents </t>
    

In [45]:
covariance=rules2.iloc[0,4]-(rules2.iloc[0,3]*rules2.iloc[0,2])
print(covariance)

0.015761194


<t>Conviction can be explained as a rule that can go wrong 1.231306 times more often if co-occurence was completely random</t>

In [46]:
conviction=(1-rules2.iloc[0,3])/(1-rules2.iloc[0,5])
print(conviction)

1.23130604637511


<h><b>6. Interpret the 5 rules with
(a) the highest confdence,
(b) the highest lift,
(c) the highest leverage, and
(d) the highest conviction.
If there are more than five meeting the required criterion, pick any five. Are any of these surprising? Comment
on the extent of their redundancy and utility.</b></h>

In [326]:
rules3a1=rules.sort_values(['support'],ascending=[False])
rules3a1
rules3a1.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(Candy Bar),(Greeting Cards),0.171005,0.146885,0.043660,0.255314,1.738191,0.018542,1.145604
3,(Greeting Cards),(Candy Bar),0.146885,0.171005,0.043660,0.297239,1.738191,0.018542,1.179626
4,(Candy Bar),(Magazine),0.171005,0.241305,0.040535,0.237040,0.982325,-0.000729,0.994410
5,(Magazine),(Candy Bar),0.241305,0.171005,0.040535,0.167982,0.982325,-0.000729,0.996367
9,(Toothpaste),(Candy Bar),0.160425,0.171005,0.039780,0.247966,1.450053,0.012347,1.102338


In [314]:
rules3a=rules.sort_values(['confidence'],ascending=[False])
rules3a
rules3a.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
43,"(Pencils, Toothpaste)",(Candy Bar),0.024560,0.171005,0.011390,0.463762,2.711980,0.007190,1.545947
30,"(Magazine, Greeting Cards)",(Candy Bar),0.036335,0.171005,0.016665,0.458649,2.682078,0.010452,1.531344
40,"(Magazine, Toothpaste)",(Candy Bar),0.031665,0.171005,0.013720,0.433286,2.533762,0.008305,1.462810
28,"(Candy Bar, Magazine)",(Greeting Cards),0.040535,0.146885,0.016665,0.411126,2.798966,0.010711,1.448723
34,"(Greeting Cards, Toothpaste)",(Candy Bar),0.032080,0.171005,0.013175,0.410692,2.401637,0.007689,1.406726


confidence and conviction gives the similar transactions though they are quite contrast to each other. Hence the minimum confidence threshold of 10% is equal to the maximum conviction threshold of 10%
leverage implies the sales of both anticidents and consequents together irrespective of their order in rule, considering the antecedents and consequencts are statistically dependant. From the results in (c) we can observe that the values of leverage are same for the records with similar antecedents and consequents but with a change in their order of transaction. Where as leverage can suffer from the rare item problem and hence we can see few differences in the rules in (b) and (c)
lift measures the redundancy of antecedents and consequents, considering them to be statistically independant. whereas, lift is susceptable to the noise. Therefore, we could observe small changes in the values of the lift for (b).

In [327]:
rules3b=rules.sort_values(['lift'], ascending=[False])
rules3b
rules3b.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
26,(Perfume),(Toothbrush),0.089960,0.067350,0.021820,0.242552,3.601370,0.015761,1.231306
27,(Toothbrush),(Perfume),0.067350,0.089960,0.021820,0.323979,3.601370,0.015761,1.346172
1,(Toothbrush),(Bow),0.067350,0.054645,0.011340,0.168374,3.081236,0.007660,1.136755
0,(Bow),(Toothbrush),0.054645,0.067350,0.011340,0.207521,3.081236,0.007660,1.176877
28,"(Candy Bar, Magazine)",(Greeting Cards),0.040535,0.146885,0.016665,0.411126,2.798966,0.010711,1.448723
31,(Greeting Cards),"(Candy Bar, Magazine)",0.146885,0.040535,0.016665,0.113456,2.798966,0.010711,1.082253
43,"(Pencils, Toothpaste)",(Candy Bar),0.024560,0.171005,0.011390,0.463762,2.711980,0.007190,1.545947
30,"(Magazine, Greeting Cards)",(Candy Bar),0.036335,0.171005,0.016665,0.458649,2.682078,0.010452,1.531344
47,"(Magazine, Toothpaste)",(Greeting Cards),0.031665,0.146885,0.011945,0.377230,2.568202,0.007294,1.369873
40,"(Magazine, Toothpaste)",(Candy Bar),0.031665,0.171005,0.013720,0.433286,2.533762,0.008305,1.462810


(Perfume)	(Toothbrush) - Redundant

In [316]:
rules3c=rules.sort_values(['leverage'], ascending=[False])
rules3c
rules3c.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(Candy Bar),(Greeting Cards),0.171005,0.146885,0.04366,0.255314,1.738191,0.018542,1.145604
3,(Greeting Cards),(Candy Bar),0.146885,0.171005,0.04366,0.297239,1.738191,0.018542,1.179626
26,(Perfume),(Toothbrush),0.089960,0.067350,0.02182,0.242552,3.601370,0.015761,1.231306
27,(Toothbrush),(Perfume),0.067350,0.089960,0.02182,0.323979,3.601370,0.015761,1.346172
9,(Toothpaste),(Candy Bar),0.160425,0.171005,0.03978,0.247966,1.450053,0.012347,1.102338


In [317]:
rules3d=rules.sort_values(['conviction'], ascending=[False])
rules3d
rules3d.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
43,"(Pencils, Toothpaste)",(Candy Bar),0.024560,0.171005,0.011390,0.463762,2.711980,0.007190,1.545947
30,"(Magazine, Greeting Cards)",(Candy Bar),0.036335,0.171005,0.016665,0.458649,2.682078,0.010452,1.531344
40,"(Magazine, Toothpaste)",(Candy Bar),0.031665,0.171005,0.013720,0.433286,2.533762,0.008305,1.462810
28,"(Candy Bar, Magazine)",(Greeting Cards),0.040535,0.146885,0.016665,0.411126,2.798966,0.010711,1.448723
34,"(Greeting Cards, Toothpaste)",(Candy Bar),0.032080,0.171005,0.013175,0.410692,2.401637,0.007689,1.406726


<h><b>7. Results of this analysis to come up with a strategic plan?</b></h>

With 1% support threshold and 10% confidence threshold, we have 50 rules that can help us in analysing and setting up the strategic plan. In this analysis metrics such as support, confidence, lift, leverage and conviction are used to formulate the plan. 

1) considering the highest support from minimum confidence threshold, which can show the most frequent item in the rule, we have 
(Candy Bar) -> (Greeting Cards)
(Greeting Cards) -> (Candy Bar)	
(Candy Bar) -> (Magazine)	
(Magazine) -> (Candy Bar)		
(Toothpaste) -> (Candy Bar)	

2) From the highest confidence we get the probability of seeing the items in consequent under the condition that the rule also contain the anticident. Hence, we have closed frequent item list among the frequent item list.
(Pencils, Toothpaste) -> (Candy Bar)
(Magazine, Greeting Cards) -> (Candy Bar)
(Magazine, Toothpaste) -> (Candy Bar)	
(Candy Bar, Magazine) -> (Greeting Cards)	
(Greeting Cards, Toothpaste) -> (Candy Bar)	

3) Finally, maximum value of lift is used to get subset of rules to make the business decision. 
(Perfume) -> (Toothbrush)
(Toothbrush) -> (Perfume)	
(Toothbrush) -> (Bow)	
(Bow) -> (Toothbrush)	
(Candy Bar, Magazine) -> (Greeting Cards)
(Greeting Cards) -> (Candy Bar, Magazine)
(Pencils, Toothpaste) -> (Candy Bar)
(Magazine, Greeting Cards) -> (Candy Bar)	
(Magazine, Toothpaste) -> (Greeting Cards)	
(Magazine, Toothpaste)- > (Candy Bar)	

Therefore, considering the rules that follow 1C2C3:
The rules that can set up a strategic plan are:
(Candy Bar, Magazine) -> (Greeting Cards)
(Greeting Cards) -> (Candy Bar, Magazine)
(Pencils, Toothpaste) -> (Candy Bar)
(Magazine, Greeting Cards) -> (Candy Bar)
(Magazine, Toothpaste)- > (Candy Bar)

